# 0. Imports

In [1]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords


## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


     |████████████████████████████████| 12.0 MB 5.2 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


# 2. Text analysis of DOJ press releases

For background, here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 

Here's the code the dataset owner used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

In [2]:
## run this code to load the unzipped json file and convert to a dataframe
## and convert some of the things from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 'components_clean']].copy()

doj.head()

,id,title,contents,date,topics_clean,components_clean
0,None,Convicted Bomb Plotter Sentenced to 30 Years,"PORTLAND, Oregon. – Mohamed Osman Mohamud, 23,...",2014-10-01T00:00:00-04:00,No topic,National Security Division (NSD)
1,12-919,$1 Million in Restitution Payments Announced t...,WASHINGTON – North Carolina’s Waccamaw River...,2012-07-25T00:00:00-04:00,No topic,Environment and Natural Resources Division
2,11-1002,$1 Million Settlement Reached for Natural Reso...,BOSTON– A $1-million settlement has been...,2011-08-03T00:00:00-04:00,No topic,Environment and Natural Resources Division
3,10-015,10 Las Vegas Men Indicted \r\nfor Falsifying V...,WASHINGTON—A federal grand jury in Las Vegas...,2010-01-08T00:00:00-05:00,No topic,Environment and Natural Resources Division
4,18-898,$100 Million Settlement Will Speed Cleanup Wor...,"The U.S. Department of Justice, the U.S. Envir...",2018-07-09T00:00:00-04:00,Environment,Environment and Natural Resources Division


## 2.1 NLP on one press release (10 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.


- Part of speech tagging- extract verbs and sort from most occurrences to least occurrences
- Named entity recognition --- what are the different organizations mentioned? how would you like to make more granular?
- Sentence level versus document-level sentiment scoring

- For sentence level scoring, print a few top positive and top negative. Does the automatic classifier seem to work?


### 2.1.1: part of speech tagging (3 points)

A. Preprocess the press release to remove all punctuation / digits (so can subset to one_word.isalpha())

B. Then, use part of speech tagging within nltk to tag all the words in that one press release with their part of speech. 

C. Finally, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print the 5 most frequent adjectives. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for .isalpha(): https://www.w3schools.com/python/ref_string_isalpha.asp
- `processtext` function here has an example of tokenizing and filtering to words where .isalpha() is true: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/06_textasdata_partII_topicmodeling_solution.ipynb 
- Part of speech tagging section of this code: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/06_textasdata_partI_textmining_examplecode.ipynb



In [4]:
doj_17 = str(doj.loc[doj['id'] == '17-1204'].contents)
pd.set_option('display.max_colwidth', None)
print(len(doj_17))
#From stack overflow: https://stackoverflow.com/questions/47361776/how-do-you-remove-punctuation-from-a-string-python
def cleaner(string):
    cleaned = [i for i in string if i.isalpha() or i.isspace()]
    return ''.join(cleaned)

doj_18 = cleaner(doj_17)
#print(doj_17)


## Code from slides
tokens = word_tokenize(doj_18) # Generate list of tokens
tokens_pos = pos_tag(tokens) # generate part of speech tags for those tokens
 
#for one_tok in tokens_pos:
   # print(one_tok)

adjectives = [one_tok[0] for one_tok in tokens_pos 
                if one_tok[1] in ["JJ", "JJS", "JJR"]]

adj_df = pd.DataFrame(adjectives)
adj_df.value_counts().head()
#adj_df.count(ascending = False).head()



9290


former        8
opioid        5
nationwide    4
addictive     3
other         3
dtype: int64

### 2.1.2 named entity recognition (3 points)


A. Using the alpha-only press release you created in the previous step, use spaCy to extract all named entities from the press release

B. Print all the named entities along with their tag

C. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these.

D. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convincted after this indictment.

**Resources**:

- Named entity recognition part of this code: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/06_textasdata_partI_textmining_examplecode.ipynb
- re.search and re.findall examples here for filtering to ones containing year (multiple approaches; some need not involve `re`): https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/04_basicregex_formerging.ipynb 

In [43]:
spacy_contents = nlp(doj_18)
for one_tok in spacy_contents.ents:
    print("Entity: " + one_tok.text + "; NER tag: " + one_tok.label_)

years_list = []
for one_tok in spacy_contents.ents:

    if (one_tok.label_ == "DATE") & ('year' in one_tok.text):
            print("Entity: " + one_tok.text + "; NER tag: " + one_tok.label_)
            years_list.append(one_tok.text)
        
print(years_list)



Entity: Insys Therapeutics Inc; NER tag: ORG
Entity: today; NER tag: DATE
Entity: Fentanyl; NER tag: PERSON
Entity: Americans; NER tag: NORP
Entity: last year; NER tag: DATE
Entity: millions; NER tag: CARDINAL
Entity: Jeff Sessions; NER tag: PERSON
Entity: This Justice Department; NER tag: ORG
Entity: Trump; NER tag: PERSON
Entity: American; NER tag: NORP
Entity: Phoenix Ariz; NER tag: ORG
Entity: the Board of Directors of Insys; NER tag: ORG
Entity: this morning; NER tag: TIME
Entity: Arizona; NER tag: GPE
Entity: RICO; NER tag: LAW
Entity: the AntiKickback Law Kapoor; NER tag: LAW
Entity: the Board; NER tag: ORG
Entity: Insys; NER tag: ORG
Entity: Phoenix; NER tag: GPE
Entity: today; NER tag: DATE
Entity: US; NER tag: GPE
Entity: District Court; NER tag: ORG
Entity: Boston; NER tag: GPE
Entity: a later date; NER tag: DATE
Entity: today; NER tag: DATE
Entity: Boston; NER tag: GPE
Entity: Insys; NER tag: ORG
Entity: December; NER tag: DATE
Entity: Kapoor Michael L Babich; NER tag: PERS

In [44]:
sentences = sent_tokenize(doj_17)

for x in years_list:
    for y in sentences:
        if x in y:
            print(x + ":" + y)

## assuming the CEO attempted to violate the anti-kickback law, 
##he is looking at five years in prison and three years on probation

last year:"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids.
three years:This investigation highlights our commitment to defending our mail system from illegal misuse and ensuring public trust in the mail.”“The U.S. Department of Veterans Affairs, Office of Inspector General will continue to aggressively investigate those that attempt to fraudulently impact programs designed to benefit our veterans and their families,” said Donna L. Neves, Special Agent in Charge of the VA OIG Northeast Field Office.The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.
three years:The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $

### 2.1.3 Sentiment analysis (4 points)

A. Use a `SentimentIntensityAnalyzer` and `polarity_scores` to score the entire press release for its sentiment (you can go back to the raw string of the press release without punctuation/digits removed)

B. Remove all named entities from the string and score the sentiment of the press release without named entities. Did the neutral score go up or down relative to the version of the press release containing named entities? Why do you think this occurred?

C. With the version of the string that removes named entities, try to split the press release into discrete sentences (hint: re.split() may be useful since it allows or conditions in the pattern you're looking for). Print the first 5 sentences of the split press release (there will not be deductions if there remain some erroneous splits; just make sure it's generally splitting)

D. Score each sentence in the split press release and print the top 5 sentences in the press release with the most negative sentiment (use the `neg` score- higher values = more negative). **Hint**: you can use pd.DataFrame to rowbind a list of dictionaries; you can then add the press release sentence for each row back as a column in that dataframe and use sort_values()                                                  
                
**Resources**:

- Sentiment analysis section of this script: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/06_textasdata_partI_textmining_examplecode.ipynb

- Discussion of using `re.split()` to split on multiple delimiters: https://stackoverflow.com/questions/4998629/split-string-with-multiple-delimiters-in-python

In [48]:
# create sentiment object
sent_obj = SentimentIntensityAnalyzer()
sentiment = sent_obj.polarity_scores(doj_17)
#print(sentiment)

ents =[]
spacy_dirty = nlp(doj_17)
for one_tok in spacy_dirty.ents:
    ents.append(one_tok.text)
#print(ents)

joined_ents = "|".join(ents)
#print(joined_ents)
subbed_doj = re.sub(joined_ents, "", doj_17)
##print(subbed_doj)

new_sent_obj = SentimentIntensityAnalyzer()
sent_new = sent_obj.polarity_scores(subbed_doj)
#print(sent_new)

## The nuetral score has decreased since removing all of the named entities.
## This is likely because these named entities have scores of their own and 
## can sway nuetrality in their use

sentence_senti_list = []
 
for sentence in sentences:
    holder = []
    holder.append(sentence)
    senter = SentimentIntensityAnalyzer()
    senti = sent_obj.polarity_scores(sentence)
    holder.append(senti["neg"])
    #print(neg)
    sentence_senti_list.append(holder)
    
#print(sentence_dict)
sentences_df = pd.DataFrame(sentence_senti_list)
sentences_df.head()
sentences_df.columns = ["content", "neg_score"]
sentences_df.sort_values(ascending = False, by = "neg_score").head()


,0,1
0,"4909 The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain.",0.382
1,"""More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids.",0.184
2,"And yet some medical professionals would rather take advantage of the addicts than try to help them,"" said Attorney General Jeff Sessions.",0.000
3,"""This Justice Department will not tolerate this.",0.000
4,We will hold accountable anyone – from street dealers to corporate executives -- who illegally contributes to this nationwide epidemic.,0.000


,content,neg_score
13,“Today's arrest and charges reflect our ongoing efforts to attack the opioid crisis from all angles.,0.494
0,"4909 The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain.",0.382
9,"The medication, called “Subsys,” is a powerful narcotic intended to treat cancer patients suffering intense breakthrough pain.",0.378
5,"And under the leadership of President Trump, we are fully committed to defeating this threat to the American people.”John N. Kapoor, 74, of Phoenix, Ariz., a current member of the Board of Directors of Insys, was arrested this morning in Arizona and charged with RICO conspiracy, as well as other felonies, including conspiracy to commit mail and wire fraud and conspiracy to violate the Anti-Kickback Law.",0.352
26,"The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.",0.351


## 2.2 sentiment scoring across many press releases (10 points)


A. Subset the press releases to those labeled with one of free topics (can just do if topic_clean == that topic rather than finding where that topic is mentioned in a longer list): Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string
- Scores the sentiment of the entire press release

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- You may want to use re.escape at some point to avoid errors relating to escape characters like ( in the press release
- I used a function + list comprehension to execute and it takes about 30 seconds on my local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample

C. Add the scores to the `doj_subset` dataframe. Sort from highest neg to lowest neg score and print the top 5 most neg.

D. With that dataframe, find the mean compound score for each of the three topics using group_by and agg. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)

**Resources**:

- Same named entity and sentiment resources as above

In [18]:
top = ['Hate Crimes', 'Civil Rights', 'Project Safe Childhood']
doj_subset = doj[doj['topics_clean'].isin(top)]


def super_senti(presser):
    named_ents = nlp(presser)

    ent_list = [one_tok.text for one_tok in named_ents.ents]
    joined_ents = "|".join(ent_list)

    subbed = re.sub(re.escape(joined_ents), "", presser)
    
    new_sent_obj = SentimentIntensityAnalyzer()
    sent_new = new_sent_obj.polarity_scores(subbed)
    return sent_new
    
#test_string = "Dartmouth College has many majors for the most ambitious students"
#super_senti(test_string)


sentiment_list = [super_senti(press_release) for press_release in doj_subset.contents]
senti_df = pd.DataFrame(sentiment_list)
senti_df.head()

senti_df.reset_index(drop = True, inplace = True)
doj_subset.reset_index(drop = True, inplace = True)


type(senti_df)



,neg,neu,pos,compound
0,0.169,0.763,0.068,-0.9893
1,0.118,0.774,0.108,-0.7003
2,0.088,0.811,0.101,0.6124
3,0.114,0.779,0.107,-0.5385
4,0.141,0.803,0.056,-0.9786


pandas.core.frame.DataFrame

In [20]:
doj_subset_with_sentiment = pd.concat([doj_subset, senti_df], axis = 1 )
len(doj_subset)
doj_subset_with_sentiment.head()

717

,id,title,contents,date,topics_clean,components_clean,neg,neu,pos,compound
0,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle",0.169,0.763,0.068,-0.9893
1,15-1522,Alabama Man Found Guilty of Aggravated Sexual Abuse of a Child,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a Department of Defense employee, were residing in Germany when they were asked to take temporary custody of a five-year-old child whose parents were deployed to Iraq with the U.S. Army. Evans sexually abused the child on multiple occasions during the 18 months that the child lived with him from May 2007 to December 2008. Trial Attorney Austin M. Berry of the Criminal Division’s Child Exploitation and Obscenity Section (CEOS) and Assistant U.S. Attorney Jacquelyn Hutzell of the Northern District of Alabama are prosecuting the case. U.S. Army Criminal Investigations Division and the FBI’s Birmingham, Alabama, Division investigated the case. This case was brought as part of Project Safe Childhood, a nationwide initiative to combat the growing epidemic of child sexual exploitation and abuse, launched in May 2006 by the Department of Justice. Led by U.S. Attorneys’ offices and CEOS, Project Safe Childhood marshals federal, state and local resources to better locate, apprehend and prosecute individuals who exploit children via the Internet, as well as to identify and rescue victims. For more information about Project Safe Childhood, please visit www.justice.gov/psc.",2

100

100

## 2.3 topic modeling (25 points)

For this question, use the `doj_subset` data that is reestricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


### 2.3.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in each of the raw strings in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem
    
B. Print the preprocessed text for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/
- Here's more condensed code with topic modeling steps: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/06_textasdata_partII_topicmodeling_solution.ipynb 
- Here's longer code with more broken-out topic modeling steps: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/06_textasdata_partII_topicmodeling_examplecode.ipynb

In [8]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]


In [21]:
pd.set_option('display.max_colwidth', None)
snow_stemmer = SnowballStemmer(language='english')
def preprocess(df, stopword_list, min_token_length = 4):
    
    new_col = []
    for x in df["contents"]:
    
        product = x.lower()
        try:

            remove_stop = [word for word in wordpunct_tokenize(product)
                          if word not in stopword_list]

            processed_string = " ".join([snow_stemmer.stem(i) 
                            for i in remove_stop if 
                            i.isalpha() and len(i) >= min_token_length])

            #print(processed_string)
            new_col.append(processed_string)
        except:
            processed_string = "error" 
            new_col.append(processed_string)
   # print(new_col)

    return(new_col)
    #df.contents.head()

## combine stopwords lists    
list_stopwords = stopwords.words("english")

list_stopwords = list_stopwords + custom_doj_stopwords

stemmed_contents = preprocess(doj_subset_with_sentiment, list_stopwords, min_token_length = 4)
doj_subset_with_sentiment["stemmed"] = stemmed_contents
doj_subset_with_sentiment.head()



,id,title,contents,date,topics_clean,components_clean,neg,neu,pos,compound,stemmed
0,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle",0.169,0.763,0.068,-0.9893,former supervisori correct offic louisiana state penitentiari angola louisiana plead guilti yesterday connect beat handcuf shackl inmat addit conspir cover misconduct falsifi offici record lie intern investig happen jame savoy marksvill louisiana admit plea hear wit offic use excess forc inmat fail interven conspir offic cover beat engag varieti obstruct act person falsifi offici prison record cover attack scotti kennedi beeb arkansa john sander marksvill louisiana previous plead guilti novemb septemb role beat cover everi citizen right process protect unreason forc correct offic violat basic constitut must held account egregi action said act general john gore continu vigor prosecut correct offic violat public trust commit crime cover violat feder crimin yesterday anoth exampl unwav commit pursu violat feder crimin law said act unit state middl louisiana corey amundson continu work close ensur investig baton roug resid agenc prosecut frederick menner middl louisiana christoph perra crimin section
1,15-1522,Alabama Man Found Guilty of Aggravated Sexual Abuse of a Child,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a Department of Defense employee, were residing in Germany when they were asked to take temporary custody of a five-year-old child whose parents were deployed to 

In [22]:
doj_16_718 = (doj_subset_with_sentiment.loc[doj_subset['id'] == '16-718'].stemmed)
doj_16_217 = (doj_subset_with_sentiment.loc[doj_subset['id'] == '16-217'].stemmed)

print(doj_16_718)
print(doj_16_217)
len(doj_subset)

632    nine count indict unseal today mississippi correct offic charg beat inmat third charg help cover indict charg lawardrick marsher robert sturdiv offic mississippi state penitentiari parchman mississippi beat includ kick punch throw victim ground marsher sturdiv charg violat right convict prison free cruel unusu punish sturdiv also charg fail interven marsher punch beat indict alleg action involv danger weapon result bodili injuri victim third offic deont pate charg along marsher sturdiv conspir cover beat indict alleg three offic submit fals report three lie convict marsher sturdiv face maximum sentenc year prison excess forc charg three offic face five year prison conspiraci fals statement charg year prison fals report charg indict mere accus defend presum innoc unless proven guilti investig jackson cooper mississippi correct prosecut robert coleman northern mississippi dana mulhaus crimin section marsher indict
Name: stemmed, dtype: object
313    reach comprehens settlement agr

717

### 2.3.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the `compound` sentiment column you added and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so most positive)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so most negative)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. What are the top 10 words for press releases in each of the three `topics_clean`?

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data

**Resources**:

- Here contains an example of applying the create_dtm function: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/06_textasdata_partII_topicmodeling_examplecode.ipynb


In [11]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), columns=vectorizer.get_feature_names())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [24]:

#doj_subset.head()
dtm_nopre = create_dtm(list_of_strings= doj_subset_with_sentiment.stemmed,
                     metadata = doj_subset_with_sentiment[['id', 'compound', 'topics_clean']].rename(columns = 
                                                        {'id': 'id_metadata', 'compound': 'compound_score'}))
dtm_nopre.head()


,index,id_metadata,compound_score,topics_clean,aaron,abandon,abbat,abbi,abbott,abdomen,...,zane,zealand,zealous,zeeman,zero,zionism,zobel,zone,zunggeemog,zwengel
0,0,17-1235,-0.9893,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,15-1522,-0.7003,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,16-213,0.6124,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,16-381,-0.5385,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,14-464,-0.9786,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:

#top_05 = dtm_nopre[dtm_nopre["compound_score"] < dtm_nopre["compound_score"].quantile(.05)]



In [27]:
top_05.shape

(36, 6870)

In [32]:
#top_terms = [col for col in dtm_nopre.columns if col not in ['id_metadata', 'index', 'compound_score', 'topics_clean']]

#top_05[top_terms].sum(axis = 0).sort_values(ascending = False).head(10)


assault     185
crime       163
victim      147
hate        126
conspir     119
offic       117
american    110
defend      106
charg       101
african     100
dtype: int64

In [ ]:
top_terms.sort_values(ascending = False).head(10)

In [35]:
def quant_words(df):
    top_terms = [col for col in df.columns if col not in ['id_metadata', 'index', 'compound_score', 'topics_clean']]

    return df[top_terms].sum(axis = 0).sort_values(ascending = False).head(10)

top_05 = dtm_nopre[dtm_nopre["compound_score"] < dtm_nopre["compound_score"].quantile(.05)]
bottom_05 = dtm_nopre[dtm_nopre["compound_score"] > dtm_nopre["compound_score"].quantile(.95)]


quant_words(top_05)
quant_words(bottom_05)




assault     185
crime       163
victim      147
hate        126
conspir     119
offic       117
american    110
defend      106
charg       101
african     100
dtype: int64

agreement     170
disabl        132
enforc        122
ensur         104
settlement    103
state         103
communiti     101
hous           90
polic          88
student        85
dtype: int64

In [37]:
hate_crimes = dtm_nopre.loc[dtm_nopre['topics_clean'] == 'Hate Crimes']
civil_rights = dtm_nopre.loc[dtm_nopre['topics_clean'] == 'Civil Rights']
psc = dtm_nopre.loc[dtm_nopre['topics_clean'] == 'Project Safe Childhood']





quant_words(hate_crimes) 
quant_words(civil_rights) 
quant_words(psc) 


victim      591
crime       557
hate        524
defend      484
prosecut    478
charg       463
sentenc     455
american    451
feder       432
guilti      430
dtype: int64

offic        637
hous         633
discrimin    616
enforc       544
disabl       532
said         497
feder        479
violat       477
state        452
court        414
dtype: int64

child          1022
exploit         701
sexual          572
safe            479
childhood       474
project         472
pornographi     452
children        423
crimin          405
prosecut        374
dtype: int64

### 2.3.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Resources**:

- Same topic modeling resources linked to above

In [39]:
tokenized_text = [wordpunct_tokenize(token) for token in 
                                      doj_subset_with_sentiment.stemmed]

### create dictionary
text_proc_dict = corpora.Dictionary(tokenized_text)
### filter dictionary- using 2% as bounds
text_proc_dict.filter_extremes(no_below = round(doj_subset_with_sentiment.shape[0]*0.02),
                             no_above = round(doj_subset_with_sentiment.shape[0]*0.98))

### create corpus from dictionary
corpus_fromdict_proc = [text_proc_dict.doc2bow(token) 
                   for token in tokenized_text]

### estimate model
n_topics = 3
ldamod_proc = gensim.models.ldamodel.LdaModel(corpus_fromdict_proc, 
                                         num_topics = n_topics, id2word=text_proc_dict, 
                                         passes=6, alpha = 'auto',
                                         per_word_topics = True, random_state = 91988)


In [40]:
topics = ldamod_proc.print_topics(num_words = 15)
for topic in topics:
    print(topic)

(0, '0.012*"victim" + 0.012*"crime" + 0.011*"prosecut" + 0.011*"hate" + 0.010*"sentenc" + 0.010*"defend" + 0.010*"charg" + 0.010*"said" + 0.010*"guilti" + 0.010*"feder" + 0.010*"american" + 0.009*"year" + 0.008*"african" + 0.008*"today" + 0.007*"investig"')
(1, '0.012*"hous" + 0.011*"offic" + 0.010*"disabl" + 0.009*"enforc" + 0.009*"discrimin" + 0.008*"said" + 0.008*"violat" + 0.008*"feder" + 0.008*"alleg" + 0.008*"state" + 0.007*"court" + 0.007*"agreement" + 0.007*"general" + 0.006*"polic" + 0.006*"today"')
(2, '0.032*"child" + 0.021*"exploit" + 0.018*"sexual" + 0.015*"safe" + 0.014*"project" + 0.014*"childhood" + 0.014*"pornographi" + 0.013*"children" + 0.013*"crimin" + 0.012*"prosecut" + 0.011*"sentenc" + 0.011*"victim" + 0.010*"ceo" + 0.010*"minor" + 0.009*"year"')


### 2.3.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset` dataframe

B. Add the topic probabilities to the `doj_subset` dataframe as columns and code each document to its highest-probability topic

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)

**Resources**:

- End of this code contains example of how to use `get_document_topics` and other steps to add topic probabilities back to data: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/06_textasdata_partII_topicmodeling_solution.ipynb

In [54]:
topic_probs_bydoc =[ldamod_proc.get_document_topics(item, minimum_probability = 0) for item in corpus_fromdict_proc]
assert len(topic_probs_bydoc) == len(doj_subset)

topic_probs_bydoc_long = pd.DataFrame([t for lst in topic_probs_bydoc for t in lst],
                                     columns = ['topic', 'probability'])


topic_probs_bydoc_long['doc_id'] = list(np.concatenate([[one_id] * n_topics for one_id in doj_subset_with_sentiment.id]).flat)
topic_probs_bydoc_long.head()

topic_probs_bydoc_wide = pd.pivot_table(topic_probs_bydoc_long, index = ['doc_id'],
                        columns = ['topic']).reset_index().reset_index(drop = True)
topic_probs_bydoc_wide.columns = ['doc_id'] + ["topic_" + str(i) for i in np.arange(0, n_topics)]
topic_probs_bydoc_wide.head()

,topic,probability,doc_id
0,0,0.303780,17-1235
1,1,0.695730,17-1235
2,2,0.000490,17-1235
3,0,0.000928,15-1522
4,1,0.000905,15-1522


,doc_id,topic_0,topic_1,topic_2
0,09-063,0.996543,0.002072,0.001386
1,09-067,0.999265,0.000440,0.000294
2,09-081,0.999487,0.000307,0.000206
3,09-1011,0.999179,0.000492,0.000329
4,09-1021,0.998045,0.001171,0.000784


In [55]:
topic_wmeta = pd.merge(topic_probs_bydoc_wide,
                      doj_subset_with_sentiment,
                      left_on = 'doc_id',
                      right_on = 'id')

topic_wmeta['toptopic'] = topic_wmeta[[col for col in topic_wmeta.columns if 
                                    "topic_" in col]].idxmax(axis=1)
topic_wmeta.head()




doc_id   topic_0   topic_1   topic_2       id  \
0   09-063  0.996543  0.002072  0.001386   09-063   
1   09-067  0.999265  0.000440  0.000294   09-067   
2   09-081  0.999487  0.000307  0.000206   09-081   
3  09-1011  0.999179  0.000492  0.000329  09-1011   
4  09-1021  0.998045  0.001171  0.000784  09-1021   

                                                                          title  \
0                   New York Man Pleads Guilty to Federal Hate Crime Conspiracy   
1  Three Men Indicted for Racially-Motivated Church Arson in Springfield, Mass.   
2                          Final Defendant Pleads Guilty to Anti-Obama Assaults   
3              Last Defendant in Tennessee Islamic Center Burning Pleads Guilty   
4                                 Two Indiana Men Plead Guilty to Cross Burning   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [64]:
#topic_wmeta.head()
pd.crosstab(topic_wmeta.topics_clean, topic_wmeta.toptopic).apply(lambda r: r/r.sum(), axis=1)

toptopic,topic_0,topic_1,topic_2
topics_clean,,,
Civil Rights,0.146179,0.847176,0.006645
Hate Crimes,1.000000,0.000000,0.000000
Project Safe Childhood,0.000000,0.006250,0.993750


In [ ]:
## civil rights has the most challenge matching data. This is likely because civil rights discussions
## oft include mentions of things like hate crimes, but also other triumphs. This is not the same with
## reports of hate crimes, where there is generally very little postive news. 

## 2.5 OPTIONAL extra credit (5 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 2.1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings. Feel free to load additional packages if needed

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?